In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "tom_slee/results/"
filename = 'all_tom_slee_listings_with_m2.csv'

In [3]:
df = pd.read_csv(path + filename, encoding='ISO-8859-1', low_memory=False)

In [4]:
# df.columns

In [5]:
def get_top_w_count(cols, frame):
    result = {}
    for c in cols:
        if 'top' in frame[c].describe() and 'freq' in frame[c].describe():
            top = frame[c].describe()['top']
            freq = frame[c].describe()['freq']
            result = {**result, **{c:{top:{'frequency':freq}}}}
    return result

In [6]:
def get_mean(cols, frame):
    result = {}
    for c in cols:
        mean = frame[c].describe()['mean']
        result = {**result, **{c:mean}}
    return result

In [7]:
def get_cols_for_type(frame, ls, nptype):
    result = []
    for l in ls:
        if frame[l].dtype is nptype:
            result.append(l)
    return result

In [8]:
string_cols = get_cols_for_type(df, df.columns, np.dtype('O'))
float_cols = get_cols_for_type(df, df.columns, np.dtype('float64'))

In [9]:
df_top_cols = get_top_w_count(string_cols, df)
df_mean_cols = get_mean(float_cols, df)

merged_cols = {}
merged_cols.update(df_top_cols)
merged_cols.update(df_mean_cols)

In [11]:
df_dropped = df.drop_duplicates('host_id')

string_cols = get_cols_for_type(df_dropped, df_dropped.columns, np.dtype('O'))
float_cols = get_cols_for_type(df_dropped, df_dropped.columns, np.dtype('float64'))

dropped_top_cols= get_top_w_count(string_cols, df_dropped)
dropped_mean_cols = get_mean(float_cols, df_dropped)

merged_dropped = {}
merged_dropped.update(dropped_top_cols)
merged_dropped.update(dropped_mean_cols)

In [12]:
import csv
def dict_to_csv(path, dictionary):
    with open(path, 'w') as f:
        w = csv.DictWriter(f, dictionary.keys())
        w.writeheader()
        w.writerow(dictionary)

In [13]:
output_path = path + "described_summary.csv"
dict_to_csv(output_path, merged_cols)

In [14]:
output_path = path +"filtered_summary.csv"
dict_to_csv(output_path, merged_dropped)

In [25]:
import json
with open(path + 'described_listings.json', 'w') as outfile:
    json.dump(df_mean_cols, outfile, ensure_ascii=False)